In [2]:
#import important libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import pandas as pd
from sklearn.cluster import SpectralClustering
from numpy import genfromtxt
import numpy as np
from scipy.linalg import null_space
from sklearn.cluster import KMeans

In [33]:
dataset_att = genfromtxt('DRUGATTR.csv', delimiter=',', skip_header = 1)
df = pd.DataFrame(dataset_att)
l1=[]
for i in range(len(df)):
    l1.append(df.loc[i][0])
df.index=[l1]
df=df.drop([0],axis=1)
#df is the attributes of people with index as their ids
df

,1,2,3
1.0,1.0,1.0,1.0
2.0,1.0,1.0,1.0
3.0,1.0,1.0,1.0
4.0,1.0,2.0,1.0
5.0,3.0,1.0,1.0
...,...,...,...
294.0,3.0,2.0,0.0
295.0,3.0,2.0,1.0
296.0,3.0,2.0,0.0
297.0,1.0,2.0,0.0


In [34]:
df.loc[1][1]

1.0    1.0
Name: 1, dtype: float64

In [35]:
network = genfromtxt('DRUGNET.csv', delimiter=',', skip_header = 1)
df_network = pd.DataFrame(network)
l2=[]
for i in range(len(df_network)):
    l2.append(df_network.loc[i][0])
df_net=df_network
df_net

,0,1,2,3,4,5,6,7,8,9,...,284,285,286,287,288,289,290,291,292,293
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,294.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289,295.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
290,296.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df_network.index = [l2]
W=df_network.drop([0],axis=1)
W=np.array(W)
W.shape

(293, 293)

In [37]:
# degree matrix
D = np.diag(np.sum(W, axis=1))
D.shape

(293, 293)

In [39]:
# laplacian matrix
L = D - W
print('laplacian matrix:')
print(L)

laplacian matrix:
[[ 2. -1.  0. ...  0.  0.  0.]
 [-1.  2.  0. ...  0.  0.  0.]
 [ 0.  0.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  1.]]


In [40]:
#finding F matrix
American=[]
Latino=[]
White=[]
for ind in df_network.index:
    if df.loc[ind[0]][1].values==2:
        American.append(ind)
    if df.loc[ind[0]][1].values==3:
        Latino.append(ind)
    else:
        White.append(ind)
n=len(American)+len(Latino)+len(White)
vsByn=[]
vsByn.append(len(American)/n)
vsByn.append(len(Latino)/n)
vsByn.append(len(White)/n)
vsByn

[0.25255102040816324, 0.39540816326530615, 0.3520408163265306]

In [41]:
#making F matrix of size nx2
F=[]
for ind in df_network.index:
    item=[]
    if df.loc[ind[0]][1].values==2:
        item.append(1-vsByn[0])
        item.append(0-vsByn[1])
    elif df.loc[ind[0]][1].values==3:
        item.append(0-vsByn[0])
        item.append(1-vsByn[1])
    else:
        item.append(0-vsByn[0])
        item.append(0-vsByn[1])
    F.append(item)
F=np.array(F)
F

array([[-0.25255102, -0.39540816],
       [-0.25255102, -0.39540816],
       [-0.25255102, -0.39540816],
       [-0.25255102, -0.39540816],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102, -0.39540816],
       [-0.25255102,  0.60459184],
       [-0.25255102, -0.39540816],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [ 0.74744898, -0.39540816],
       [ 0.74744898, -0.39540816],
       [ 0.74744898, -0.39540816],
       [ 0.74744898, -0.39540816],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [ 0.74744898, -0.39540816],
       [ 0.74744898, -0.39540816],
       [ 0.74744898, -0.39540816],
       [-0.25255102,  0.60459184],
       [-0.25255102,  0.60459184],
       [-0.25255102,

In [42]:
F=F.T
F.shape

(2, 293)

In [13]:
Z = null_space(F)
Z.shape

(293, 291)

In [14]:
U=Z.T.dot(L).dot(Z)
U.shape

(291, 291)

In [15]:
e, v = np.linalg.eig(U)
e = e.real
v = v.real
v

array([[ 0.00087161,  0.00096716,  0.00096716, ...,  0.00407634,
         0.00047714,  0.00047714],
       [ 0.00086886,  0.00089256,  0.00089256, ...,  0.00596255,
         0.0017605 ,  0.0017605 ],
       [ 0.00050408, -0.00201303, -0.00201303, ..., -0.00096312,
        -0.00127704, -0.00127704],
       ...,
       [ 0.00050408, -0.00177189, -0.00177189, ...,  0.00424873,
        -0.0010925 , -0.0010925 ],
       [ 0.00088259,  0.0012326 ,  0.0012326 , ..., -0.00812204,
        -0.00924225, -0.00924225],
       [ 0.00050408, -0.00177189, -0.00177189, ..., -0.00094805,
        -0.00128054, -0.00128054]])

In [16]:
Y=[]
clusters=4
for i in range(clusters):
    Y.append(v[i])
Y=np.array(U)
Y

array([[ 1.00542747e+00,  9.68664107e-03, -7.05931583e-03, ...,
        -1.13184911e-02,  1.39458164e-02, -7.05931583e-03],
       [ 5.42746578e-03,  2.00968664e+00, -7.05931583e-03, ...,
        -1.13184911e-02,  1.39458164e-02, -7.05931583e-03],
       [-1.38504751e-03, -6.34548021e-03,  1.01018888e+00, ...,
         1.51493165e-02, -1.13059129e-02,  1.01888838e-02],
       ...,
       [-1.38504751e-03, -6.34548021e-03,  1.01888838e-02, ...,
         1.51493165e-02, -1.13059129e-02,  1.01888838e-02],
       [ 5.42746578e-03,  9.68664107e-03, -7.05931583e-03, ...,
        -1.13184911e-02,  1.39458164e-02, -7.05931583e-03],
       [-1.38504751e-03, -6.34548021e-03,  1.01888838e-02, ...,
         1.51493165e-02, -1.13059129e-02,  1.01018888e+00]])

In [17]:
H=Z.dot(Y)
H

array([[ 0.00592288,  0.01068972, -0.11249834, ..., -0.11726518,
         0.01545656, -0.11249834],
       [-0.14900259, -0.14423575,  0.16789946, ...,  0.16313262,
        -0.13946891,  0.16789946],
       [ 0.99380822, -0.00142494,  0.0031309 , ..., -0.00163594,
         0.0033419 ,  0.0031309 ],
       ...,
       [ 0.00801665,  0.00291219, -0.00375191, ...,  0.00135255,
        -0.00219227, -0.00375191],
       [-0.00619178, -0.00142494,  0.0031309 , ..., -0.00163594,
         0.0033419 ,  0.0031309 ],
       [ 0.00801665,  0.00291219, -0.00375191, ...,  0.00135255,
        -0.00219227,  0.99624809]])

In [18]:
#creating a k-means object
k_Means_Obj=KMeans(n_clusters=4)
import time
start = time.time()
k_Means_Obj.fit(H)
end = time.time()
clusters=k_Means_Obj.cluster_centers_

In [19]:
clusters

array([[-0.00422584, -0.00235206,  0.00205634, ...,  0.00018256,
        -0.00047828,  0.00205634],
       [ 0.00801665,  0.00291219, -0.00375191, ...,  0.00135255,
        -0.00219227, -0.00375191],
       [ 0.00801665,  0.00291219, -0.00375191, ...,  0.00135255,
        -0.00219227, -0.00375191],
       [ 0.00801665,  0.00291219, -0.00375191, ...,  0.00135255,
        -0.00219227, -0.00375191]])

In [20]:
final=k_Means_Obj.fit_predict(H)

In [21]:
final

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [22]:
cluster1=[]
cluster2=[]
cluster3=[]
cluster4=[]
for i in range(len(final)):
    if final[i]==0:
        cluster1.append(i)
    elif final[i]==1:
        cluster2.append(i)
    elif final[i]==2:
        cluster3.append(i)
    elif final[i]==3:
        cluster4.append(i)
    elif final[i]==4:
        cluster5.append(i)
clusters = [cluster1,cluster2,cluster3,cluster4]
clusters = np.array(clusters)
clusters

C:\Users\NEHA\AppData\Local\Temp/ipykernel_1896/1945950731.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  clusters = np.array(clusters)


array([list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 